<h1>Ford Care Classifier</h1>

Modelo desenvolvido a fim de complementar o guesser, tentando prever o tipo de alarme que vai ocorrer

In [2]:
import pandas as pd
pd.set_option('display.max_columns',None)

from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import pickle

def get_train_data():
    df = pd.read_csv("../data/ford_classifier_fuel.csv", index_col='Unnamed: 0')
    return df

def format_guesser_data(df):
    
    df = df.join(pd.get_dummies(df.MARK))
    df = df.join(pd.get_dummies(df.MODEL))
    df.drop(['MARK','MODEL','VEHICLE_ID'],inplace=True,axis=1)
    df.AUTOMATIC = df.AUTOMATIC.replace({'s':1, 'n':0})
    df.DTC_NUMBER = df.DTC_NUMBER.replace({'MIL is OFF1 codes':1, 'MIL is OFF0 codes':0})
    df['SHORT TERM FUEL TRIM BANK 1'].fillna(df['SHORT TERM FUEL TRIM BANK 1'].mean(), inplace=True)

    for c in df.columns:
        if df[c].isna().sum() > 0:
            df[c] = df[c].fillna(df[c].mean())
    return df

def train_classifier(df, model):
    x = df[['CAR_YEAR', 'AIR_INTAKE_TEMP',
           'ENGINE_COOLANT_TEMP', 'ENGINE_LOAD', 'ENGINE_POWER', 'ENGINE_RPM',
           'INTAKE_MANIFOLD_PRESSURE', 'SPEED', 'THROTTLE_POS']]

    y = df['ERROR CODE']

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

    
    return model.fit(x_train, y_train)

def export_classifier_pickle(clf):
    filename = '../app/ford_care/ford_classifier.sav'
    pickle.dump(clf, open(filename, 'wb'))
    

if __name__ == "__main__":
    df = get_train_data()
    df = format_guesser_data(df)
    knn = KNeighborsClassifier(n_neighbors=len(df['ERROR CODE'].unique()))
    knn_trained = train_classifier(df, knn)
    export_classifier_pickle(knn_trained)

In [39]:
df['$ERROR CODE'] = knn.predict(df[t.columns])
print('Ford_Classifier acurácia = ' + str(metrics.accuracy_score(df['ERROR CODE'], df['$ERROR CODE'])))

Ford_Classifier acurácia = 0.9541808083924714
